In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from tqdm import tqdm

import sqlalchemy as sqla
from sqlalchemy import create_engine, Column, and_
from sqlalchemy.orm import Session

from ultrack.core.database import NodeDB
from ultrack.core.export.utils import solution_dataframe_from_sql

import sys
sys.path.append('..')
from tracks_interactions.db.db_model import Base, CellDB, TrackDB
from tracks_interactions.db.db_functions import add_track_ids_to_tracks_df, calculate_cell_signals

In [2]:
# create a new database

new_db_path = r'D:\kasia\tracking\E6_exp\sample_data\cells_database_expand.db'

engine = create_engine(f'sqlite:///{new_db_path}')

# creates a table
Base.metadata.create_all(engine) 

In [3]:
# get engine for the original database

org_db_path = r'D:\kasia\tracking\E6_exp\code\gardener_20_dev\data.db'
engine_org = create_engine(f'sqlite:///{org_db_path}')

In [4]:
# get a solution in a form of a dataframe

df = solution_dataframe_from_sql(f'sqlite:///{org_db_path}')
df = add_track_ids_to_tracks_df(df)

df.reset_index(inplace=True)
df

c:\Users\kmk280\.conda\envs\tracking\lib\site-packages\numba\typed\typeddict.py:39: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  return d[key]


,id,parent_id,t,z,y,x,track_id,parent_track_id,root
0,1000017,-1,0,0.0,213.0,4549.0,1,-1,1.0
1,1000020,-1,0,0.0,225.0,3607.0,4,-1,4.0
2,1000021,-1,0,0.0,231.0,5027.0,5,-1,5.0
3,1000028,-1,0,0.0,270.0,3647.0,8,-1,8.0
4,1000029,-1,0,0.0,289.0,3631.0,9,-1,9.0
...,...,...,...,...,...,...,...,...,...
2021053,239010184,238010283,238,0.0,8311.0,4620.0,69589,-1,69589.0
2021054,239010185,238010284,238,0.0,8312.0,4995.0,64351,-1,64351.0
2021055,239010186,238010287,238,0.0,8317.0,4130.0,73597,73595,73595.0
2021056,239010187,238010289,238,0.0,8319.0,4072.0,73596,73595,73595.0


In [5]:
df.columns

Index(['id', 'parent_id', 't', 'z', 'y', 'x', 'track_id', 'parent_track_id',
       'root'],
      dtype='object')

## Create a cells table

In [6]:
# that has to be changed to operate on the original database
# because at the moment objects not assigned to a track are not saved in the database
# the consideration is what if multiple segmentations were given to ultrack and
# there are multiple possible objects for a single cell ???

def add_cell(row):

        global session
        global session_db_org
        
        cell = CellDB(id = row['id'],
                    t =row['t'],
                    track_id = row['track_id'],
                    row = row['y'],
                    col = row['x'])
        
        # get a mask of this cell
        cell_obj = session_db_org.query(NodeDB).filter(NodeDB.id==row['id']).first()

        cell.mask = cell_obj.pickle.mask
        
        cell.bbox_0 = int(cell_obj.pickle.bbox[0])
        cell.bbox_1 = int(cell_obj.pickle.bbox[1])
        cell.bbox_2 = int(cell_obj.pickle.bbox[2])
        cell.bbox_3 = int(cell_obj.pickle.bbox[3])

        session.add(cell)

In [7]:
# create a table of cells
# exp6 - ~ 15 min

tqdm.pandas(desc="Progress")

session_db_org = Session(engine_org)
session = Session(engine)

df.progress_apply(add_cell, axis=1)

session.commit()

session_db_org.close()
session.close()

Progress: 100%|██████████| 2021058/2021058 [13:24<00:00, 2512.98it/s]


### Add signals to the cells table

In [8]:
import dask.array as da

In [9]:
ch0_path = r'D:\kasia\tracking\E6_exp\E6_C0.zarr'
ch1_path = r'D:\kasia\tracking\E6_exp\E6_C1.zarr'

ch0_da = da.from_zarr(ch0_path,1)
ch1_da = da.from_zarr(ch1_path,1)

In [25]:
type(ch0_da)==da.core.Array

True

In [10]:
# for exp6 around 25 min

session = Session(engine)

for frame in tqdm(range(ch0_da.shape[0])):

    cells = session.query(CellDB).filter(CellDB.t==frame).all()
    ch0 = ch0_da[frame].compute()
    ch1 = ch1_da[frame].compute()

    for cell in cells:

        # Calculate cell measurements for each cell
        new_signals = calculate_cell_signals(cell, [ch0, ch1])
        
        # Update the signals field with the new JSON data
        cell.signals = new_signals
        
    # Commit changes to the database
    session.commit()

session.close()

100%|██████████| 241/241 [24:03<00:00,  5.99s/it]


## Create a tracks table

In [11]:
df_tracks = df.groupby(['track_id','parent_track_id' ,'root']).agg({'t':['min','max']})
df_tracks.reset_index(inplace=True)
df_tracks.columns = ['_'.join(col).strip('_') for col in df_tracks.columns.values]
df_tracks

,track_id,parent_track_id,root,t_min,t_max
0,1,-1,1.0,0,73
1,2,1,1.0,74,160
2,3,1,1.0,74,169
3,4,-1,4.0,0,154
4,5,-1,5.0,0,10
...,...,...,...,...,...
75013,75014,75012,75012.0,239,240
75014,75015,-1,75015.0,238,240
75015,75016,-1,75016.0,238,240
75016,75017,-1,75017.0,238,240


In [12]:
def add_track(row):

        global session
        
        track = TrackDB(track_id = row['track_id'],
                        parent_track_id = row['parent_track_id'],
                        root = row['root'],
                        t_begin = row['t_min'],
                        t_end = row['t_max'])
        

        session.add(track)

In [13]:
# create a table of tracks

session = Session(engine)  

df_tracks.apply(add_track, axis=1)

session.commit()

session.close()

## Tests

In [14]:
with Session(engine) as session:

    results = session.query(CellDB.t, CellDB.signals['ch1_nuc'])\
        .filter(CellDB.track_id == 40)\
        .order_by(CellDB.t)\
        .all()

results

[(72, 1124.8367816092),
 (73, 974.61690647482),
 (74, 1056.7583497053),
 (75, 1048.63026819923),
 (76, 934.575959933222),
 (77, 870.22629969419),
 (78, 902.058441558442),
 (79, 871.124213836478),
 (80, 799.590395480226),
 (81, 896.310177705977),
 (82, 878.967032967033),
 (83, 898.824193548387),
 (84, 849.671775223499),
 (85, 868.837282780411),
 (86, 842.46015037594),
 (87, 880.224324324324),
 (88, 821.526315789474),
 (89, 791.246110325318),
 (90, 768.794466403162),
 (91, 782.233196159122),
 (92, 813.235807860262),
 (93, 920.645328719723),
 (94, 789.628180039139),
 (95, 767.321828358209),
 (96, 907.497478991597),
 (97, 797.595716198126),
 (98, 915.130136986301),
 (99, 1009.13223140496)]